## RNN

In [55]:
import numpy as np
import optuna
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from optuna import Trial, Study
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm
from utils.training import train, best_torch_device, tutorial_train
from torchinfo import summary

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
device = best_torch_device()
# device = torch.device('cpu')
print(best_torch_device())

mps


In [57]:
X_train_valid = np.load("data/X_train_valid.npy")
y_train_valid = np.load("data/y_train_valid.npy")
person_train_valid = np.load("data/person_train_valid.npy")

X_train_aug = np.load("data/generated/frequency/X_train_augmented.npy")
y_train_aug = np.load("data/generated/frequency/y_train_augmented.npy")

X_test = np.load("data/X_test.npy")
y_test = np.load("data/y_test.npy")
person_test = np.load("data/person_test.npy")

y_train_valid -= 769
y_train_aug -= 769
y_test -= 769

X_train = X_train_valid[:1777]
y_train = y_train_valid[:1777]
X_valid = X_train_valid[1777:]
y_valid = y_train_valid[1777:]


In [58]:
X_train.shape

(1777, 22, 1000)

In [59]:
class RNN(nn.Module):

    def __init__(self):
        super(RNN, self).__init__()

        conv1_filter_num = 32
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=22, out_channels=conv1_filter_num, kernel_size=(5, 1), padding=(5, 0)),
            nn.ELU(),
            nn.AvgPool2d(kernel_size=(3, 1)),
            nn.BatchNorm2d(conv1_filter_num),
            nn.Dropout(0.3)
        )

        conv2_filter_num = 128
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=conv1_filter_num, out_channels=conv2_filter_num, kernel_size=(5, 1), padding=(5, 0)),
            nn.ELU(),
            nn.AvgPool2d(kernel_size=(3, 1)),
            nn.BatchNorm2d(conv2_filter_num),
            nn.Dropout(0.3)
        )

        conv3_filter_num = 128
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=conv2_filter_num, out_channels=conv3_filter_num, kernel_size=(5, 1), padding=(5, 0)),
            nn.ELU(),
            nn.AvgPool2d(kernel_size=(3, 1)),
            nn.BatchNorm2d(conv3_filter_num),
            nn.Dropout(0.3)
        )

        conv_out_filter_size = 19
        self.flatten = nn.Flatten(start_dim=2)
        self.linear1 = nn.Linear(in_features=conv_out_filter_size, out_features=conv_out_filter_size)

        self.flatten_cnn = nn.Flatten(start_dim=1)
        self.linear_cnn = nn.Linear(in_features=conv_out_filter_size*conv3_filter_num, out_features=4)

        self.tdd = nn.Linear(in_features=1000, out_features=50)

        conv_rnn_filter_num = 50
        self.conv_rnn = nn.Sequential(
            nn.Conv2d(in_channels=22, out_channels=conv_rnn_filter_num, kernel_size=(64, 1), padding=(5, 0)),
            nn.ELU(),
            nn.AvgPool2d(kernel_size=(7, 1)),
            nn.BatchNorm2d(conv_rnn_filter_num),
            nn.Dropout(0.3)
        )

        self.num_layers = 3
        self.hidden_size = 30
        self.lstm = nn.LSTM(input_size=conv3_filter_num, hidden_size=self.hidden_size, dropout=0.5, num_layers=self.num_layers)
        # self.lstm = nn.LSTM(input_size=conv_rnn_filter_num, hidden_size=self.hidden_size, dropout=0.5, num_layers=self.num_layers)
        
        self.linear2 = nn.Linear(in_features=self.hidden_size, out_features=4) # (N, 4)

        self.softmax = nn.Softmax(dim=1) # the dim corresponds to num_output_classes=4

    def forward(self, x):
        """
        x should have shape (N, H, L) = (N, 22, 1000), where
            L = sequence length
            N = batch size
            H = input size
        """
        (N, H, L) = x.shape

        x = x.view((-1, 22, 1000, 1))
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        # x = self.linear1(x)
        # x = self.flatten_cnn(x)
        # x = self.linear_cnn(x)

        # x = self.tdd(x)
        # x = x.view((-1, 22, 1000, 1))
        # x = self.conv_rnn(x)
        # x = self.flatten(x)
        x = x.permute(2, 0, 1) # permute to (L=time, N, H=channel_in)
        # print(x.shape)

        # device = torch.device('mps')
        # h0 = torch.randn(self.num_layers, N, self.hidden_size).to(device)
        # c0 = torch.randn(self.num_layers, N, self.hidden_size).to(device)
        # x, _ = self.lstm(x, (h0, c0))
        x, _ = self.lstm(x)
        x = self.linear2(x[-1]) # take the last time step
        
        x = self.softmax(x)
        return x

In [71]:
class RNN(nn.Module):

    def __init__(self):
        super(RNN, self).__init__()

        conv1_filter_num = 32
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=22, out_channels=conv1_filter_num, kernel_size=(5, 1), padding=(5, 0)),
            nn.ELU(),
            nn.AvgPool2d(kernel_size=(3, 1)),
            nn.BatchNorm2d(conv1_filter_num),
            nn.Dropout(0.3)
        )

        conv2_filter_num = 128
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=conv1_filter_num, out_channels=conv2_filter_num, kernel_size=(5, 1), padding=(5, 0)),
            nn.ELU(),
            nn.AvgPool2d(kernel_size=(3, 1)),
            nn.BatchNorm2d(conv2_filter_num),
            nn.Dropout(0.3)
        )

        conv3_filter_num = 128
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=conv2_filter_num, out_channels=conv3_filter_num, kernel_size=(5, 1), padding=(5, 0)),
            nn.ELU(),
            nn.AvgPool2d(kernel_size=(3, 1)),
            nn.BatchNorm2d(conv3_filter_num),
            nn.Dropout(0.3)
        )

        conv_out_filter_size = 19
        self.flatten = nn.Flatten(start_dim=2)
        self.linear1 = nn.Linear(in_features=conv_out_filter_size, out_features=conv_out_filter_size)

        self.flatten_cnn = nn.Flatten(start_dim=1)
        self.linear_cnn = nn.Linear(in_features=conv_out_filter_size*conv3_filter_num, out_features=4)

        self.tdd = nn.Linear(in_features=1000, out_features=50)

        conv_rnn_filter_num = 50
        self.conv_rnn = nn.Sequential(
            nn.Conv2d(in_channels=22, out_channels=conv_rnn_filter_num, kernel_size=(64, 1), padding=(5, 0)),
            nn.ELU(),
            nn.AvgPool2d(kernel_size=(7, 1)),
            nn.BatchNorm2d(conv_rnn_filter_num),
            nn.Dropout(0.3)
        )

        self.num_layers = 3
        self.hidden_size = 30
        self.lstm = nn.LSTM(input_size=conv3_filter_num, hidden_size=self.hidden_size, dropout=0.5, num_layers=self.num_layers)
        # self.lstm = nn.LSTM(input_size=conv_rnn_filter_num, hidden_size=self.hidden_size, dropout=0.5, num_layers=self.num_layers)
        
        self.linear2 = nn.Linear(in_features=self.hidden_size, out_features=4) # (N, 4)

        self.softmax = nn.Softmax(dim=1) # the dim corresponds to num_output_classes=4

    def forward(self, x):
        """
        x should have shape (N, H, L) = (N, 22, 1000), where
            L = sequence length
            N = batch size
            H = input size
        """
        (N, H, L) = x.shape

        x = x.view((-1, 22, 1000, 1))
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        # x = self.linear1(x)
        # x = self.flatten_cnn(x)
        # x = self.linear_cnn(x)

        # x = self.tdd(x)
        # x = x.view((-1, 22, 1000, 1))
        # x = self.conv_rnn(x)
        # x = self.flatten(x)
        x = x.permute(2, 0, 1) # permute to (L=time, N, H=channel_in)
        # print(x.shape)

        # device = torch.device('mps')
        # h0 = torch.randn(self.num_layers, N, self.hidden_size).to(device)
        # c0 = torch.randn(self.num_layers, N, self.hidden_size).to(device)
        # x, _ = self.lstm(x, (h0, c0))
        x, _ = self.lstm(x)
        x = self.linear2(x[-1]) # take the last time step
        
        x = self.softmax(x)
        return x

In [72]:
bsz = 128
sample_device = torch.device('mps')
sample_model = RNN().to(sample_device)
test_input = torch.randn(128, 22, 1000).to(sample_device)
print(sample_model(test_input).shape)
summary(sample_model, (bsz, 22, 1000), device=device)



torch.Size([128, 4])


Layer (type:depth-idx)                   Output Shape              Param #
RNN                                      [128, 4]                  130,712
├─Sequential: 1-1                        [128, 32, 335, 1]         --
│    └─Conv2d: 2-1                       [128, 32, 1006, 1]        3,552
│    └─ELU: 2-2                          [128, 32, 1006, 1]        --
│    └─AvgPool2d: 2-3                    [128, 32, 335, 1]         --
│    └─BatchNorm2d: 2-4                  [128, 32, 335, 1]         64
│    └─Dropout: 2-5                      [128, 32, 335, 1]         --
├─Sequential: 1-2                        [128, 128, 113, 1]        --
│    └─Conv2d: 2-6                       [128, 128, 341, 1]        20,608
│    └─ELU: 2-7                          [128, 128, 341, 1]        --
│    └─AvgPool2d: 2-8                    [128, 128, 113, 1]        --
│    └─BatchNorm2d: 2-9                  [128, 128, 113, 1]        256
│    └─Dropout: 2-10                     [128, 128, 113, 1]        --
├─

In [73]:
bsz = 128

train_loader = DataLoader(TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train)), shuffle=True, batch_size=bsz)
train_aug_loader = DataLoader(TensorDataset(torch.from_numpy(X_train_aug), torch.from_numpy(y_train_aug)), shuffle=True, batch_size=bsz)
val_loader = DataLoader(TensorDataset(torch.from_numpy(X_valid), torch.from_numpy(y_valid)), shuffle=False, batch_size=bsz)
test_loader = DataLoader(TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test)), shuffle=False, batch_size=bsz)

print(len(train_loader), len(train_aug_loader), len(val_loader), len(test_loader))

print(X_train_aug.shape)

14 42 3 4
(5331, 22, 1000)


In [74]:
model = RNN().to(device)
print(model)
num_chans, sequence_length = 22, 1000  # Image dimensions
test_input = torch.randn(bsz, num_chans, sequence_length).to(device)
print(model(test_input).shape)
# pred = model(test_input)[0]
# print(nn.CrossEntropyLoss()(pred, torch.from_numpy(y_train[0:1])))

RNN(
  (conv1): Sequential(
    (0): Conv2d(22, 32, kernel_size=(5, 1), stride=(1, 1), padding=(5, 0))
    (1): ELU(alpha=1.0)
    (2): AvgPool2d(kernel_size=(3, 1), stride=(3, 1), padding=0)
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Dropout(p=0.3, inplace=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 128, kernel_size=(5, 1), stride=(1, 1), padding=(5, 0))
    (1): ELU(alpha=1.0)
    (2): AvgPool2d(kernel_size=(3, 1), stride=(3, 1), padding=0)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Dropout(p=0.3, inplace=False)
  )
  (conv3): Sequential(
    (0): Conv2d(128, 128, kernel_size=(5, 1), stride=(1, 1), padding=(5, 0))
    (1): ELU(alpha=1.0)
    (2): AvgPool2d(kernel_size=(3, 1), stride=(3, 1), padding=0)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Dropout(p=0.3, inplace=False)
  )
  (flatten): Flatten(start_dim=2,

In [75]:
cel_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0)
num_epochs = 100
loader = train_aug_loader
train_acc_hist = []
val_acc_hist = []
for epoch_idx in tqdm(range(num_epochs)):
    # Set model to train mode - useful for layers such as BatchNorm or Dropout whose behaviors change between train/eval
    model.train()
    train_count = 0
    train_correct_count = 0
    for batch_idx, (train_x, train_y) in enumerate(loader):
        train_x = train_x.float().to(device)
        train_y = train_y.long().to(device)
        optimizer.zero_grad()
        logits = model(train_x)
        loss = cel_loss(logits, train_y)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            y_hat = torch.argmax(logits, dim=-1)
            train_correct_count += torch.sum(y_hat == train_y, axis=-1)
            train_count += train_x.size(0)

    train_acc = train_correct_count / train_count
    train_acc_hist.append(train_acc)

    model.eval()
    val_count = 0
    val_correct_count = 0
    with torch.no_grad():
        for idx, (val_x, val_y) in enumerate(val_loader):
            val_x = val_x.float().to(device)
            val_y = val_y.long().to(device)
            logits = model(val_x).detach()
            y_hat = torch.argmax(logits, dim=-1)
            val_correct_count += torch.sum(y_hat == val_y, axis=-1)
            val_count += val_x.size(0)
    val_acc = val_correct_count / val_count
    val_acc_hist.append(val_acc)

    print('Train acc: {:.3f}, Val acc: {:.3f}'.format(train_acc, val_acc))

  0%|          | 0/100 [00:00<?, ?it/s]

Train acc: 0.258, Val acc: 0.243
Train acc: 0.261, Val acc: 0.272
Train acc: 0.276, Val acc: 0.254
Train acc: 0.274, Val acc: 0.269
Train acc: 0.252, Val acc: 0.269
Train acc: 0.260, Val acc: 0.278
Train acc: 0.255, Val acc: 0.257
Train acc: 0.258, Val acc: 0.260
Train acc: 0.261, Val acc: 0.251
Train acc: 0.252, Val acc: 0.263
Train acc: 0.252, Val acc: 0.260
Train acc: 0.257, Val acc: 0.266
Train acc: 0.258, Val acc: 0.266
Train acc: 0.251, Val acc: 0.272
Train acc: 0.252, Val acc: 0.269
Train acc: 0.253, Val acc: 0.269
Train acc: 0.245, Val acc: 0.263
Train acc: 0.252, Val acc: 0.275
Train acc: 0.255, Val acc: 0.257
Train acc: 0.255, Val acc: 0.260
Train acc: 0.254, Val acc: 0.275
Train acc: 0.252, Val acc: 0.260
Train acc: 0.252, Val acc: 0.234
Train acc: 0.263, Val acc: 0.257
Train acc: 0.255, Val acc: 0.263
Train acc: 0.255, Val acc: 0.272
Train acc: 0.254, Val acc: 0.243
Train acc: 0.252, Val acc: 0.257
Train acc: 0.255, Val acc: 0.260
Train acc: 0.247, Val acc: 0.254
Train acc:

KeyboardInterrupt: 

In [49]:
len(X_train_aug)

10662

In [37]:
all_predictions = []
all_labels = []

model.eval()
val_count = 0
val_correct_count = 0
with torch.no_grad():
    for idx, (val_x, val_y) in enumerate(test_loader):
        val_x = val_x.float().to(device)
        val_y = val_y.long().to(device)
        logits = model(val_x).detach()
        y_hat = torch.argmax(logits, dim=-1)
        val_correct_count += torch.sum(y_hat == val_y, axis=-1)
        val_count += val_x.size(0)
val_acc = val_correct_count / val_count

print("Test Accuracy:", val_acc.item())

Test Accuracy: 0.40406322479248047


In [36]:
print(model)

RNN(
  (conv1): Sequential(
    (0): Conv2d(22, 64, kernel_size=(64, 1), stride=(1, 1), padding=(5, 0))
    (1): ELU(alpha=1.0)
    (2): MaxPool2d(kernel_size=(3, 1), stride=(3, 1), padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Dropout(p=0.3, inplace=False)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 200, kernel_size=(64, 1), stride=(1, 1), padding=(5, 0))
    (1): ELU(alpha=1.0)
    (2): MaxPool2d(kernel_size=(3, 1), stride=(3, 1), padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Dropout(p=0.3, inplace=False)
  )
  (conv3): Sequential(
    (0): Conv2d(200, 200, kernel_size=(64, 1), stride=(1, 1), padding=(5, 0))
    (1): ELU(alpha=1.0)
    (2): MaxPool2d(kernel_size=(3, 1), stride=(3, 1), padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(200, eps=1e-05, momentum=0.1, affine=True, track_runnin